<a href="https://colab.research.google.com/github/PN1019/StockExchanges/blob/main/StockMarketWebStatics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_datareader as DataReader
from datetime import date
import pandas_datareader.data as web
import matplotlib.figure as figure


In [ ]:
 pip install plotly

In [ ]:
 pip install cufflinks

In [ ]:
import cufflinks as cf
import plotly.graph_objs as go
import plotly as py #plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
cf.go_offline()

In [ ]:
    init_notebook_mode(connected=True)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving listings.xlsx to listings.xlsx


In [ ]:
nasdaq = pd.read_excel('listings.xlsx', sheet_name='nasdaq',na_values='n/a')

In [ ]:
market_cap = nasdaq['Market Capitalization'].div(10**6)


In [ ]:
print('Mean of Market Capitalization: \n' ,market_cap.mean())
print('Median of Market Capitalization: \n' ,market_cap.median())
print('Mode of Market Capitalization: \n' ,market_cap.mode())


Mean of Market Capitalization: 
 3180.7126214953823
Median of Market Capitalization: 
 225.9684285
Mode of Market Capitalization: 
 0    0.0
dtype: float64


## Quantiles

In [ ]:
median = market_cap.quantile(.5)


In [ ]:
median == market_cap.median()

True

In [ ]:
# quantile in  market capitalization
quantiles = market_cap.quantile([.25,.75])
print(quantiles)

print(quantiles[.75]-quantiles[.25])


0.25     43.375930
0.75    969.905207
Name: Market Capitalization, dtype: float64
926.5292771575


In [ ]:
deciles = np.arange(start=.1,stop=.91,step=.1)

In [ ]:
deciles

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [ ]:
 market_cap.quantile(deciles)


0.1       4.884565
0.2      26.993382
0.3      65.714547
0.4     124.320644
0.5     225.968428
0.6     402.469678
0.7     723.163197
0.8    1441.071134
0.9    3671.499558
Name: Market Capitalization, dtype: float64


**Visualize quantiles with bar chart**


In [ ]:
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)
# Alternative Way to Show each plot using this function instead of renderer="colab"

In [ ]:
#from plotly.offline import iplot
#import plotly.graph_objs as go

#enable_plotly_in_cell()

data=[go.Bar(x=deciles,y=market_cap.quantile(deciles),name="NASDAQ Market Capitalization (million USD)")]
layout=go.Layout(title="NASDAQ Market Capitalization (million USD)")
fig=go.Figure(data=data,layout=layout)
#fig.show()
fig.show(renderer="colab")

**EDA Stats**:

In [ ]:
market_cap.describe()


count      3167.000000
mean       3180.712621
std       25471.038707
min           0.000000
25%          43.375930
50%         225.968428
75%         969.905207
max      740024.467000
Name: Market Capitalization, dtype: float64

In [ ]:
market_cap.describe(percentiles=np.arange(.1,.91,.1))


count      3167.000000
mean       3180.712621
std       25471.038707
min           0.000000
10%           4.884565
20%          26.993382
30%          65.714547
40%         124.320644
50%         225.968428
60%         402.469678
70%         723.163197
80%        1441.071134
90%        3671.499558
max      740024.467000
Name: Market Capitalization, dtype: float64

### Decade treasury trend distribution

In [ ]:
ty10 = web.DataReader('DGS10','fred',date(1962,1,1))

In [ ]:
ty10.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15478 entries, 1962-01-02 to 2021-04-29
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DGS10   14817 non-null  float64
dtypes: float64(1)
memory usage: 241.8 KB


**Aggregating data by Category**

**Group data by sector**

In [ ]:
nasdaq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3167 entries, 0 to 3166
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Stock Symbol           3167 non-null   object 
 1   Company Name           3167 non-null   object 
 2   Last Sale              3165 non-null   float64
 3   Market Capitalization  3167 non-null   float64
 4   IPO Year               1386 non-null   float64
 5   Sector                 2767 non-null   object 
 6   Industry               2767 non-null   object 
dtypes: float64(3), object(4)
memory usage: 173.3+ KB


In [ ]:
nasdaq['market_cap_m'] = nasdaq['Market Capitalization'].div(1e6)

In [ ]:
nasdaq = nasdaq.drop('Market Capitalization', axis=1) # Drop column

nasdaq_by_sector = nasdaq.groupby('Sector') # create groupby onject


In [ ]:
for sector, data in nasdaq_by_sector:
    print(sector, data.market_cap_m.mean())


Basic Industries 724.8999338580088
Capital Goods 1511.2373727809975
Consumer Durables 839.8026066270018
Consumer Non-Durables 3104.0512055241334
Consumer Services 5582.344174819412
Energy 826.607607854727
Finance 1044.0902047996321
Health Care 1758.709196756265
Miscellaneous 3445.655934557965
Public Utilities 2357.8653150733353
Technology 10883.43421354678
Transportation 2869.660006726413


In [ ]:
# easy way

mcap_by_sector = nasdaq_by_sector.market_cap_m.mean()
print(mcap_by_sector)

Sector
Basic Industries           724.899934
Capital Goods             1511.237373
Consumer Durables          839.802607
Consumer Non-Durables     3104.051206
Consumer Services         5582.344175
Energy                     826.607608
Finance                   1044.090205
Health Care               1758.709197
Miscellaneous             3445.655935
Public Utilities          2357.865315
Technology               10883.434214
Transportation            2869.660007
Name: market_cap_m, dtype: float64


In [ ]:
data=[go.Bar(x=nasdaq_by_sector.market_cap_m.mean(),y=nasdaq['Sector'].unique(),name="NASDAQ = Avg. Market Cap by Sector",orientation='h',marker=dict(
        color='rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=0)
))]
layout=go.Layout(title="NASDAQ Avg. Market Cap by Sector",xaxis_title="USD (In Millions)",
    yaxis=dict(title="Sectors"))

fig=go.Figure(data=data,layout=layout)
fig.show(renderer="colab")
#all sector are listed now but data is not showing the avg. mcap by sector
 

**Aggregation Summary**

In [ ]:
nasdaq_by_sector.mean()

,Last Sale,IPO Year,market_cap_m
Sector,,,
Basic Industries,21.597679,2000.766667,724.899934
Capital Goods,26.188681,2001.324675,1511.237373
Consumer Durables,24.363391,2003.222222,839.802607
Consumer Non-Durables,25.749565,2000.609756,3104.051206
Consumer Services,34.917318,2004.104575,5582.344175
Energy,15.496834,2008.034483,826.607608
Finance,29.644242,2010.321101,1044.090205
Health Care,19.462531,2009.240409,1758.709197
Miscellaneous,46.094369,2004.333333,3445.655935


***Aggreagation By Category***

In [ ]:
nasdaq_by_sector.market_cap_m.agg(['size','mean']).sort_values('size')

,size,mean
Sector,,
Transportation,52,2869.660007
Energy,66,826.607608
Public Utilities,66,2357.865315
Basic Industries,78,724.899934
Consumer Durables,88,839.802607
Miscellaneous,89,3445.655935
Consumer Non-Durables,103,3104.051206
Capital Goods,172,1511.237373
Consumer Services,348,5582.344175


**Different Statics by Column**

In [ ]:
nasdaq_by_sector.agg({'market_cap_m':'size','IPO Year':'median'})


,market_cap_m,IPO Year
Sector,,
Basic Industries,78,1996.5
Capital Goods,172,2000.0
Consumer Durables,88,2002.5
Consumer Non-Durables,103,1998.0
Consumer Services,348,2005.0
Energy,66,2011.0
Finance,627,2014.0
Health Care,645,2014.0
Miscellaneous,89,2005.0


In [ ]:
nasdaq_by_sector = nasdaq.groupby(['Sector','IPO Year'])

nasdaq_by_sector.market_cap_m.mean()


Sector            IPO Year
Basic Industries  1972.0       877.240005
                  1973.0      1445.697371
                  1986.0      1396.817381
                  1988.0        24.847526
                  1990.0       434.808483
                                 ...     
Transportation    2006.0      1397.129447
                  2007.0       413.900806
                  2009.0       552.445919
                  2011.0      3711.638317
                  2013.0       125.740421
Name: market_cap_m, Length: 280, dtype: float64

In [ ]:
# In the Sorted order
sector_size = nasdaq.groupby('Sector').size()
order = sector_size.sort_values(ascending=False)
order.head()

Sector
Health Care          645
Finance              627
Technology           433
Consumer Services    348
Capital Goods        172
dtype: int64

In [ ]:
data=[go.Bar(x=nasdaq['Sector'].unique(),y=sector_size,textangle=45)]
layout=go.Layout(title="Observations Per Sector",yaxis_title="Sector Size",xaxis_title="Sectors")
fig=go.Figure(data=data,layout=layout)
#fig.show()
fig.show(renderer="colab") 

In [ ]:
trace=[]
recent_ipos = nasdaq[nasdaq['IPO Year'] > 2014]
recent_ipos['IPO Year'] = recent_ipos['IPO Year'].astype(int)
yyyy = recent_ipos['IPO Year'].unique().tolist()[-3:] #slice function
for y in yyyy:
    tmp_df = recent_ipos[recent_ipos['IPO Year'] == y].groupby('Sector')['market_cap_m'].sum()
    trace.append(go.Bar(x=tmp_df.index, y=tmp_df, name=y))
    
# Change the bar mode
fig = go.Figure(trace)
fig.update_layout(barmode='group',title="NASDAQ Market Capitalization Sectorwise in Last Three IPO Year (million USD)",yaxis_title="Market Capital(In Millions USD)", xaxis_title="Sectors")
fig.show(renderer="colab")
# need to show data tips with modified text

In [ ]:
# Active companies only
nasdaq = nasdaq[nasdaq.market_cap_m > 0]

#Outlier threshold
outliers = nasdaq.market_cap_m.quantile(.9)

# Removing Outliers
nasdaq = nasdaq[nasdaq.market_cap_m < outliers]


In [ ]:
traces=[]
traces.append(go.Box(
            y=nasdaq['market_cap_m'],
            x=nasdaq['Sector'],
            notched=True, # used notched shape
            boxpoints='all',
            jitter=0.5,
            whiskerwidth=0.2,
            marker_color='rgb(7,40,89)',
    line_color='rgb(7,40,89)',
    marker=dict(
        color='rgb(8,81,156)',
        outliercolor='rgba(219, 64, 82, 0.6)',
        line=dict(
            outliercolor='rgba(219, 64, 82, 0.6)',
            outlierwidth=2))
    
        ))
layout = go.Layout(
    title='Boxplot Showing Quartiles and Outliers in Different Sectors',
    yaxis=dict(
        range=[0,10.50],
        autorange=True,
        showgrid=True,
        zeroline=True,
        dtick=5,
        gridcolor='rgb(255, 255, 255)',
        gridwidth=1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False
)
fig = go.Figure(data=traces, layout=layout)
fig.show(renderer="colab")
# issue with y axis here why the scale is not showing here
#issue is reported on https://github.com/plotly/plotly.py/issues/3264